<a href="https://colab.research.google.com/github/rajesh0305/Image-Processing-And-Computer-Vision-/blob/main/Dropout_%26_Regularizors%26Layers_Normalization%26Early_stoppingf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install wandb
import wandb
from wandb.integration.keras import WandbMetricsLogger ,WandbModelCheckpoint
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import keras
from tensorflow import keras

In [8]:
sweep_config = {
    'method': 'grid',
'metric' : {
    'name': 'val_accuracy',
    'goal':'maximize'
},
    'parameters': {
         'batch_size' :{'values':[8]},
         'learning_rate' :{'values' :[0.001]},
         'hidden_nodes': {'values':[128]},
         'img_size': {'values':[16]},
         'epochs':{'values':[10]}
    }
}
sweep_id = wandb.sweep(sweep_config,project='5-flowers-new-v3')

Create sweep with ID: it4v2rad
Sweep URL: https://wandb.ai/rajesh8368568776-national-institute-of-technology-karnat/5-flowers-new-V3/sweeps/it4v2rad


In [9]:
def train():
  with wandb.init() as run:
    config = wandb.config

    # Constants
    IMG_HEIGHT = config.img_size
    IMG_WIDTH = config.img_size
    IMG_CHANNELS = 3
    CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

    def read_and_decode(filename, resize_dims):
        img_bytes = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.resize(img, resize_dims)
        return img

    def parse_csvline(csv_line):
        record_default = ["", ""]
        filename, label_string = tf.io.decode_csv(csv_line, record_default)

        img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])

        # Convert label string to an integer index
        label = tf.where(tf.equal(CLASS_NAMES, label_string))[0, 0]

        return img, label

    # Define datasets
    train_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    eval_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    #architecture
    regularizers = tf.keras.regularizers.l1_l2(0,0.1)
    # model = keras.Sequential([
    #     keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
    #     keras.layers.Dense(config.hidden_nodes,kernel_regularizer=regularizers, activation="relu"),
    #     keras.layers.Dropout(0.5), # means 50% of the neurons not used
    #     keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
    # ])

    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),

        keras.layers.Dense(config.hidden_nodes),
        keras.layers.BatchNormalization(),
        keras.layers.Activation("relu"),

        keras.layers.Dense(len(CLASS_NAMES)),
        keras.layers.BatchNormalization(),
        keras.layers.activation("softmax")
    ])

    model.compile(
          optimizer="adam",
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
          metrics=["accuracy"]
      )

    model.fit(
      train_dataset,
      validation_data=eval_dataset,
      epochs=config.epochs,
      callbacks=[WandbMetricsLogger(log_freq=5),
                 tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience = 3,restore_best_weights = True)
                 ]
      )

In [10]:
wandb.agent(sweep_id,function=train)

wandb: Agent Starting Run: gfeybqi5 with config:
wandb: 	batch_size: 8
wandb: 	epochs: 10
wandb: 	hidden_nodes: 128
wandb: 	img_size: 16
wandb: 	learning_rate: 0.001


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
    413/Unknown 191s 454ms/step - accuracy: 0.2698 - loss: 7.7370

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


413/413 ━━━━━━━━━━━━━━━━━━━━ 213s 507ms/step - accuracy: 0.2699 - loss: 7.7272 - val_accuracy: 0.3541 - val_loss: 1.5559
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 150s 362ms/step - accuracy: 0.2986 - loss: 1.5892 - val_accuracy: 0.3324 - val_loss: 1.5416
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 112s 271ms/step - accuracy: 0.2938 - loss: 1.5911 - val_accuracy: 0.3622 - val_loss: 1.5398
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 143s 347ms/step - accuracy: 0.3015 - loss: 1.5824 - val_accuracy: 0.3676 - val_loss: 1.5328
Epoch 5/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 89s 216ms/step - accuracy: 0.3031 - loss: 1.5715 - val_accuracy: 0.4270 - val_loss: 1.4932
Epoch 6/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 100s 242ms/step - accuracy: 0.3041 - loss: 1.5901 - val_accuracy: 0.4297 - val_loss: 1.5115
Epoch 7/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 90s 219ms/step - accuracy: 0.2984 - loss: 1.5790 - val_accuracy: 0.3730 - val_loss: 1.5170
Epoch 8/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 88s 213ms/step - accuracy: 0.3152 - loss: 1.5719 

batch/accuracy,▂▂▄▄▄▅▅▄▅█▅▅▅▅▅▅▅▅▅▅▅▅▄▆▅▁▅▆▆▆▄▄▄▄▄▅▅▆▆▆
batch/batch_step,▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇████
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▃▇▅▆▆▇▇█▁█
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▃▁▃▄██▄▁▂▁
epoch/val_loss,█▆▆▅▁▃▄▆▇▇
batch/accuracy,0.31509


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
